In [ ]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip3 install google-api-python-client langchain anthropic transformers
from googleapiclient.discovery import build
import concurrent.futures
import os
import re
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatAnthropic
from transformers import AutoModelForCausalLM, AutoTokenizer
import time

GOOGLE_API_KEY = os.getenv("GOOGLE_SEARCH")  # Replace with your Google Search API key
GOOGLE_CSE_ID = os.getenv("GOOGLE_SEARCH_ENGINE_ID")  # Replace with your Custom Search Engine ID
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")  # Replace with your Anthropic API key

In [7]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu116
!pip3 install google-api-python-client langchain anthropic transformers[cuda116]

from googleapiclient.discovery import build
import concurrent.futures
import os
import re
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatAnthropic
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch
import time

GOOGLE_API_KEY = os.getenv("GOOGLE_SEARCH")  # Replace with your Google Search API key
GOOGLE_CSE_ID = os.getenv("GOOGLE_SEARCH_ENGINE_ID")  # Replace with your Custom Search Engine ID
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")  # Replace with your Anthropic API key

def remove_first_line(test_string):
    if test_string.startswith("Here") and test_string.split("\n")[0].strip().endswith(":"):
        return re.sub(r'^.*\n', '', test_string, count=1)
    return test_string

def generate_text(llm, prompt, max_new_tokens=1024, temperature=0.7, max_retries=3, retry_delay=1):
    for i in range(max_retries):
        try:
            response = llm(prompt, max_new_tokens=max_new_tokens, temperature=temperature)
            response = remove_first_line(response[0]['generated_text'].strip())
            print(f"Response: {response}")
            match = re.search(r'\[(.*?)\]', response)
            if match:
                try:
                    items_str = match.group(1)
                    if items_str.startswith("'") and items_str.endswith("'"):
                        items_str = items_str[1:-1]
                    items = [item.strip().strip('"').strip("'") for item in items_str.split(',')]
                    print(f"Items: {items}")
                    return items
                except (SyntaxError, ValueError) as e:
                    print(f"Error parsing items: {e}")
                    pass
        except Exception as e:
            print(f"Error generating text (attempt {i+1}/{max_retries}): {e}")
            if i < max_retries - 1:
                time.sleep(retry_delay)
    return []

def search_web(search_term, api_key, cse_id):
    try:
        service = build("customsearch", "v1", developerKey=api_key)
        return service.cse().list(q=search_term, cx=cse_id).execute()
    except Exception as e:
        print(f"Error searching web: {e}")
        return None

def generate_subtopic_report(llm, research_topic, subtopic, search_cache, max_new_tokens=1024, temperature=0.7, max_workers=5):
    all_queries = []
    search_data = []
    
    print(f"Generating initial search queries for subtopic: {subtopic}...")
    initial_queries_prompt = f"Generate 5 search queries to gather information on the subtopic '{subtopic}' of the research topic '{research_topic}'. Return your queries in a Python-parseable list. Return nothing but the list. Do so in one line. Start your response with [\""
    initial_queries = generate_text(llm, initial_queries_prompt, max_new_tokens=max_new_tokens)
    all_queries.extend(initial_queries)

    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        search_results = list(executor.map(lambda query: search_cache.setdefault(query, search_web(query, GOOGLE_API_KEY, GOOGLE_CSE_ID)), initial_queries))
    search_data.extend(search_results)

    print(f"Generating initial report for subtopic: {subtopic}...")
    report_prompt_1 = f"When writing your report, make it incredibly detailed, thorough, specific, and well-structured. Use Markdown for formatting. "
    report_prompt_2 = f"Analyze the following search data and generate a comprehensive report on the subtopic '{subtopic}' of the research topic '{research_topic}':\n\n{str(search_data)}"
    report_prompt = report_prompt_1 + report_prompt_2
    report = llm(report_prompt, max_new_tokens=max_new_tokens, temperature=temperature)[0]['generated_text']

    print(f"Analyzing report and generating additional searches for subtopic: {subtopic}...")
    analysis_prompt_1 = f"Analyze the following report on the subtopic '{subtopic}' of the research topic '{research_topic}' and identify areas that need more detail or further information:\n\n{report}\n\n"
    analysis_prompt_2 = "---\n\nHere are all the search queries you have used so far for this subtopic:\n\n"
    analysis_prompt_3 = f"{str(all_queries)}\n\n---\n\nGenerate 3 new and unique search queries to fill in the gaps and provide more detail on the identified areas. Return your queries in a Python-parseable list. Return nothing but the list. Do so in one line. Start your response with [\""
    analysis_prompt = analysis_prompt_1 + analysis_prompt_2 + analysis_prompt_3
    additional_queries = generate_text(llm, analysis_prompt, max_new_tokens=max_new_tokens)
    all_queries.extend(additional_queries)

    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        additional_search_results = list(executor.map(lambda query: search_cache.setdefault(query, search_web(query, GOOGLE_API_KEY, GOOGLE_CSE_ID)), additional_queries))
    search_data.extend(additional_search_results)

    print(f"Updating report with additional information for subtopic: {subtopic}...")
    update_prompt_1 = f"Update the following report on the subtopic '{subtopic}' of the research topic '{research_topic}' by incorporating the new information from the additional searches. Keep all existing information... only add new information:\n\n{report}\n\n---\n\nAdditional search data:\n\n{str(additional_search_results)}\n\n---\n\n"
    update_prompt_2 = "Generate an updated report that includes the new information and provides more detail in the identified areas. Use Markdown for formatting."
    update_prompt = update_prompt_1 + update_prompt_2
    report = llm(update_prompt, max_new_tokens=max_new_tokens, temperature=temperature)[0]['generated_text']
    
    print(f"Final report generated for subtopic: {subtopic}!")
    return report

def generate_comprehensive_report(chat, research_topic, subtopic_reports):
    print("Generating comprehensive report...")
    comprehensive_report_prompt_1 = f"Generate a comprehensive report on the research topic '{research_topic}' by combining the following reports on various subtopics:\n\n{subtopic_reports}\n\n"
    comprehensive_report_prompt_2 = "---\n\nEnsure that the final report is well-structured, coherent, and covers all the important aspects of the research topic. Make sure that it includes EVERYTHING in each of the reports, in a better structured, more info-heavy manner. Nothing -- absolutely nothing, should be left out. If you forget to include ANYTHING from any of the previous reports, you will face the consequences. Include a table of contents. Leave nothing out. Use Markdown for formatting."
    comprehensive_report_prompt = comprehensive_report_prompt_1 + comprehensive_report_prompt_2
    comprehensive_report = chat([comprehensive_report_prompt]).content
    return comprehensive_report

def is_valid_subtopic_list(subtopic_list, research_topic):
    if not isinstance(subtopic_list, list):
        print("Subtopic checklist is not a list.")
        return False
    for subtopic in subtopic_list:
        if not isinstance(subtopic, str):
            print(f"Subtopic '{subtopic}' is not a string.")
            return False
        if subtopic.lower() == research_topic.lower() or subtopic.lower().startswith(f"{research_topic.lower()}:"):
            print(f"Subtopic '{subtopic}' is the same as the research topic.")
            return False
    return True

def main(research_topic, model_name, max_new_tokens=1024, max_subtopics=5, max_workers=5):
    if not GOOGLE_API_KEY:
        print("Google Search API key not found. Please set the GOOGLE_SEARCH environment variable.")
        return
    if not GOOGLE_CSE_ID:
        print("Google Custom Search Engine ID not found. Please set the GOOGLE_SEARCH_ENGINE_ID environment variable.")
        return
    if not ANTHROPIC_API_KEY:
        print("Anthropic API key not found. Please set the ANTHROPIC_API_KEY environment variable.")
        return
   
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
    llm = pipeline("text-generation", model=model, tokenizer=tokenizer, device=device)
   
    subtopic_checklist_prompt = f"Generate a detailed checklist of subtopics to research for the topic '{research_topic}'. Return your checklist as a valid Python list, with each subtopic enclosed in single quotes and separated by commas. Maximum {max_subtopics} sub-topics. Start your response with ['{research_topic}:']"
    subtopic_checklist = generate_text(llm, subtopic_checklist_prompt, max_new_tokens=max_new_tokens)
    print(f"Generated subtopic checklist: {subtopic_checklist}")
    if not is_valid_subtopic_list(subtopic_checklist, research_topic):
        print("Invalid subtopic checklist generated. Please try again with a different research topic.")
        return
    
    print(f"Research Topic: {research_topic}")
    print(f"Subtopic Checklist: {subtopic_checklist}")
    
    search_cache = {}
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        subtopic_reports = list(executor.map(lambda subtopic: generate_subtopic_report(llm, research_topic, subtopic, search_cache, max_new_tokens=max_new_tokens, temperature=0.7, max_workers=max_workers), subtopic_checklist))
   
    chat = ChatAnthropic(temperature=0, anthropic_api_key=ANTHROPIC_API_KEY, model_name="claude-3-opus-20240229")
    comprehensive_report = generate_comprehensive_report(chat, research_topic, "\n\n".join(subtopic_reports))
    print("Comprehensive report generated!")
    
    with open("comprehensive_report.txt", "w") as file:
        file.write(comprehensive_report)
    print("Comprehensive report saved as 'comprehensive_report.txt'.")

if __name__ == "__main__":
    research_topic = input("Enter the research topic: ")
    model_name = input("Enter the name of the HuggingFace model to use: ")
    main(research_topic, model_name, max_new_tokens=1024, max_subtopics=5, max_workers=5)

ValidationError: 2 validation errors for HuggingFaceHub
token
  extra fields not permitted (type=value_error.extra)
__root__
  Did not find huggingfacehub_api_token, please add an environment variable `HUGGINGFACEHUB_API_TOKEN` which contains it, or pass `huggingfacehub_api_token` as a named parameter. (type=value_error)